# Group 6
# Apurv (291049)
# Arshpreet Singh Bagga(291013)
# Anuj Sharma (291070)
# Jumbish S Dang (291032)
# Bikramadityya Guho (291017)

In [16]:

import numpy as np 
import pandas as pd 


import os
path =  "/content/drive/MyDrive/Colab Notebooks/CTR"
os.listdir(path)


['train.gz', 'sampleSubmission.gz', 'test.gz']

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import random
import warnings
warnings.filterwarnings('ignore')

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
n = 40428967  #total number of records in the clickstream data 
sample_size = 3000000
skip_values = sorted(random.sample(range(1,n), n-sample_size))

with gzip.open('../content/drive/MyDrive/Colab Notebooks/CTR/test.gz') as f:
    train = pd.read_csv(f,skiprows = skip_values)
train['hour'] = pd.to_datetime(train['hour'],format = '%y%m%d%H')

In [20]:
train.head()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10002000217531288531,2014-10-31,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,dc17d849,ac9ad752,1,0,23642,320,50,2709,3,39,-1,23
1,10002614093820007721,2014-10-31,1005,0,85f751fd,c4e18dd6,50e219e0,03528b27,2347f47a,8ded1f7a,df62db53,346c1c70,81892725,1,0,17016,320,50,1873,3,39,-1,23
2,10003460544097147762,2014-10-31,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,2d4180b2,8a4875bd,1,0,6392,320,50,547,3,47,100084,23
3,10007361632582633559,2014-10-31,1002,0,9e8e8d09,16a36ef3,50e219e0,ecad2386,7801e8d9,07d7df22,52260ad2,6118f3f9,dca31af0,0,0,22700,320,50,2624,0,35,-1,221
4,10007689954273878529,2014-10-31,1005,0,85f751fd,c4e18dd6,50e219e0,98fed791,d9b5648e,0f2161f8,a99f214a,592568ac,aad45b01,1,0,19743,320,50,2264,3,425,100000,61


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339541 entries, 0 to 339540
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                339541 non-null  uint64        
 1   hour              339541 non-null  datetime64[ns]
 2   C1                339541 non-null  int64         
 3   banner_pos        339541 non-null  int64         
 4   site_id           339541 non-null  object        
 5   site_domain       339541 non-null  object        
 6   site_category     339541 non-null  object        
 7   app_id            339541 non-null  object        
 8   app_domain        339541 non-null  object        
 9   app_category      339541 non-null  object        
 10  device_id         339541 non-null  object        
 11  device_ip         339541 non-null  object        
 12  device_model      339541 non-null  object        
 13  device_type       339541 non-null  int64         
 14  devi

In [22]:
train.isnull().sum()

id                  0
hour                0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
dtype: int64

In [23]:
train.describe(include = 'all')

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,3.395410e+05,339541,339541.000000,339541.000000,339541,339541,339541,339541,339541,339541,339541,339541,339541,339541.000000,339541.000000,339541.000000,339541.000000,339541.000000,339541.000000,339541.000000,339541.00000,339541.000000,339541.000000
unique,NaN,24,NaN,NaN,1591,1554,19,1746,93,24,38152,199401,3771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2014-10-31 09:00:00,NaN,NaN,85f751fd,c4e18dd6,50e219e0,ecad2386,7801e8d9,07d7df22,a99f214a,6b9769f2,1f0bc64f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,22007,NaN,NaN,127437,130433,138369,212104,220552,213866,292365,1463,15666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2014-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2014-10-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.233663e+18,NaN,1004.988299,0.211382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.009507,0.377383,21713.568155,320.217211,59.555088,2469.317066,1.305442,185.42045,53502.568497,92.153089
std,5.323791e+18,NaN,0.930577,0.475535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428189,0.900338,3765.423330,28.794259,46.722098,475.502606,1.305401,262.84778,49942.043013,84.043357
min,1.282261e+13,NaN,1001.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,375.000000,120.000000,20.000000,112.000000,0.000000,33.00000,-1.000000,13.000000
25%,4.626283e+18,NaN,1005.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,21611.000000,320.000000,50.000000,2480.000000,0.000000,35.00000,-1.000000,23.000000


In [ ]:
X = train.drop('click', axis=1)
y = train.click

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [ ]:
num_cols = X.select_dtypes(include = ['int','float']).columns.tolist()
categorical_cols = X.select_dtypes(include = ['object']).columns.tolist()
print(num_cols)
print(categorical_cols)

## Hashing


In [ ]:
for col in categorical_cols:
	X_train[col] = X_train[col].apply(lambda x: hash(x))
    
for col in categorical_cols:
    X_test[col] = X_test[col].apply(lambda x:hash(x))

## Scaling the Numerical Values

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train[num_cols] = std.fit_transform(X_train[num_cols])
X_test[num_cols] = std.transform(X_test[num_cols])

In [ ]:
X_train['user_info'] = X_train.device_ip + X_train.device_model + X_train.device_id
X_train = X_train.drop(['device_id','device_ip','device_model','id','hour'],axis=1)
    
X_train['device_info'] = X_train.device_type + X_train.banner_pos + X_train.device_conn_type
X_train = X_train.drop(['banner_pos','device_conn_type','device_type'],axis=1)

X_test['user_info'] = X_test.device_ip + X_test.device_model + X_test.device_id
X_test = X_test.drop(['device_id','device_ip','device_model','id','hour'],axis=1)
    
X_test['device_info'] = X_test.device_type + X_test.banner_pos + X_test.device_conn_type
X_test = X_test.drop(['banner_pos','device_conn_type','device_type'],axis=1)

## Model - Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 10)
tree.fit(X_train,y_train)
print('Train Score:',tree.score(X_train,y_train))
print('Test Score:',tree.score(X_test,y_test))

In [ ]:
from sklearn.metrics import roc_curve,confusion_matrix,precision_score,recall_score,roc_auc_score
y_score = tree.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc_score = roc_auc_score(y_test,y_score[:,1])
print(roc_auc_score)

In [ ]:
y_pred = tree.predict(X_test)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
print("Precision: %s, Recall: %s" %(precision, recall))

In [ ]:
matrix = confusion_matrix(y_test,y_pred)
tn, fp, fn, tp = matrix.ravel()
print(matrix)

In [ ]:
for max_depth_val in [2, 3, 5, 10, 15, 20]:
    clf = DecisionTreeClassifier(max_depth = max_depth_val)
    print("Evaluating tree with max_depth = %s" %(max_depth_val))
    y_pred = tree.fit(X_train,y_train).predict(X_test) 
    print("Confusion matrix: ")
    print(confusion_matrix(y_test, y_pred))
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average = 'weighted')
    print("Precision: %s, Recall: %s" %(prec, recall))

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
for max_depth_val in [3, 5, 10]:
    k_fold = KFold(n_splits = 4)
    clf = DecisionTreeClassifier(max_depth = max_depth_val)
    print("Evaluating Decision Tree for max_depth = %s" %(max_depth_val))
    y_pred = tree.fit(X_train, y_train).predict(X_test) 
  
    cv_precision = cross_val_score(clf, X_train, y_train, cv = k_fold, scoring = 'recall_weighted')
    precision = recall_score(y_test, y_pred, average = 'weighted')
    print("Cross validation Recall: %s" %(cv_precision.mean()))
    print("Test Recall: %s" %(precision.mean()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [10], 
    'n_estimators':[2,5,10,20,50],
    'min_samples_split': [2, 3, 4],
    'max_features' : ['log2']
}

rf = RandomForestClassifier(random_state=42)

rf_search = RandomizedSearchCV(estimator = rf, param_distributions=param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, n_iter = 10)

model = rf_search.fit(X_train,y_train)

In [ ]:
rf_search.best_params_

In [ ]:
clf_rf = RandomForestClassifier(n_estimators= 20,
 min_samples_split= 2,
 max_features= 'log2',
 max_depth= 10,
 bootstrap= True,
 random_state=42)
clf_rf.fit(X_train,y_train)

In [ ]:
y_test_pred = clf_rf.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
prec = precision_score(y_test, y_test_pred, average = 'weighted')
recall = recall_score(y_test, y_test_pred, average = 'weighted')
print("Precision: %s, Recall: %s" %(prec, recall))

In [ ]:
rf_prob = clf_rf.predict_proba(X_test)
rf_fpr,rf_tpr,_ = roc_curve(y_test,rf_prob[:,1])

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,confusion_matrix,precision_score,recall_score,roc_auc_score
params = {
    "objective": "binary:logistic",
    "booster" : "gbtree",
    "eval_metric": "logloss",
    "eta":0.1,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "silent": 1,
}
xgclf=xgb.XGBClassifier(**params)
xgclf.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='logloss',
        verbose=False)
xgpred=xgclf.predict_proba(X_test)

In [ ]:
from sklearn import metrics 
#y_score = xgclf.predict_proba(x_test)
#fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc_score = metrics.roc_auc_score(y_test,xgpred[:,1])
print(roc_auc_score)
print(xgclf.score(X_test,y_test))
xgb_fpr,xgb_tpr,_ = roc_curve(y_test,xgpred[:,1])

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
# Set up k-fold
k_fold = KFold(n_splits = 5)

# Evaluate precision and recall for each fold
precision = cross_val_score(
  xgclf, X_train, y_train, cv = k_fold, scoring = 'precision_weighted')
recall = cross_val_score(
  xgclf, X_train, y_train, cv = k_fold, scoring = 'recall_weighted')
print("Precision scores: %s" %(precision.mean())) 
print("Recall scores: %s" %(recall.mean()))
print(k_fold)

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.style.use('seaborn')
plt.plot(fpr,tpr,linestyle = '--',color = 'green',label='Decision Tree')
plt.plot(rf_fpr,rf_tpr,linestyle = '--',color = 'yellow',label='Random Forest')
plt.plot(xgb_fpr,xgb_tpr,linestyle = '--',color = 'orange',label='XGBoost')
plt.plot(p_fpr,p_tpr,linestyle='--',color = 'blue')
plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('AUC-ROC Curve')